In [1]:
from app.database_redis import keys
from app.database_redis.connection import get_redis_client
from app.services.audio.audio import AudioSlicer
from app.services.audio.redis import Diarisation, Diarizer, Meeting, best_covering_connection
from app.settings import settings
from datetime import datetime,timezone

check_and_process_connections_interval_sec: 5.0


In [2]:
from pathlib import Path
import random

def ls(self):
    """
    List the contents of the directory.
    Returns:
        List[Path]: A list of Paths in the directory.
    """
    if not self.is_dir():
        raise NotADirectoryError(f"{self} is not a directory")
    
    return list(self.iterdir())
Path.ls = ls
from app.services.audio.audio import AudioSlicer
import json
import pandas as pd
import numpy as np

/tmp/ipykernel_829/910237743.py:17: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
from app.database_redis.connection import get_redis_client

In [4]:
from app.services.audio.redis import Diarisation, Diarizer, Meeting, best_covering_connection

In [5]:
redis_client = await get_redis_client(settings.redis_host, settings.redis_port, settings.redis_password)

In [6]:
diarizer = Diarizer(redis_client)
meeting_id = await diarizer.pop_inprogress()

In [ ]:
meeting

In [7]:
audiofiles = Path('/audio').ls()

In [8]:
import random

In [20]:
results =[]
for _ in range(10):
    path = random.choice(audiofiles)
    result = {}
    result['path']= path
    try:
        audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 30)
        passed = True
    except:
        passed = False

    result['passed'] = passed
        
    with open(path,'rb') as f:
        result['bytes'] = f.read()[:500]

    results.append(result)

In [21]:
results = pd.DataFrame(results)

In [22]:
results[results['passed']]

,path,passed,bytes
0,/audio/58b2fc09-99fc-4242-8246-0298b1c436f4.webm,True,b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B...
1,/audio/c0c29ee3-bdea-4e8e-a14f-a8df87c517e7.webm,True,"b""\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B..."
3,/audio/f7734c60-e38b-4217-b559-2318f6be494d.webm,True,b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B...
4,/audio/22c1dd13-93bd-4881-a10f-25dada2153b2.webm,True,"b""\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B..."
7,/audio/4daafbbd-e346-4837-974a-bdc036f87266.webm,True,b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B...
8,/audio/54d378a2-75a1-4614-8aa5-d6ca07146e2c.webm,True,"b""\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B..."
9,/audio/7497d02a-2910-49e4-a758-ec59f473e6f8.webm,True,b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B...


In [25]:
results[~results['passed']].iloc[0]

path       /audio/491a596a-aecd-497d-888b-a4ba514f76f4.webm
passed                                                False
bytes     b'\x1aE\xdf\xa3\xef\xbf\xbdB\xef\xbf\xbd\xef\x...
Name: 2, dtype: object

In [29]:
path = results[~results['passed']].iloc[0]['path']

In [33]:
audio_slicer = await AudioSlicer.from_ffmpeg_slice(path, 0, 100)

CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 5.1.4-0+deb12u1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 12 (Debian 12.2.0-14)
  configuration: --prefix=/usr --extra-version=0+deb12u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --ena

In [21]:
with open(path,'rb') as f:
    r = f.read()[:500]

In [22]:
r

b'\x1aE\xdf\xa3\x9fB\x86\x81\x01B\xf7\x81\x01B\xf2\x81\x04B\xf3\x81\x08B\x82\x84webmB\x87\x81\x04B\x85\x81\x02\x18S\x80g\x01\xff\xff\xff\xff\xff\xff\xff\x15I\xa9f\x99*\xd7\xb1\x83\x0fB@M\x80\x86ChromeWA\x86Chrome\x16T\xaek\xbf\xae\xbd\xd7\x81\x01s\xc5\x87\x10\x12\xd63F\xdb\x7f\x83\x81\x02\x86\x86A_OPUSc\xa2\x93OpusHead\x01\x02\x00\x00\x80\xbb\x00\x00\x00\x00\x00\xe1\x8d\xb5\x84G;\x80\x00\x9f\x81\x02bd\x81 \x1fC\xb6u\x01\xff\xff\xff\xff\xff\xff\xff\xe7\x81\x00\xa3A\xf5\x81\x00\x00\x80\xff\x83\x02\x02\xff\xfe\xff\xfe\x7f\xdf\x00\xd6\x06\xa6\xf0|O\xb0?\xb3?\xf2,\xbbFU\xd8\x03\x11~\x88\xf4\\S\xb5\xd4\x10\x81\xfd\xe2\n\xed)\xdd\xae\x88\xef\x80x\xaeRA\xf0x\xcd\xec\x9b\xfe{[\x80\xf7\xdc&\x8a\xb9\xbd\xddm\xcc\xfbp\xeb\x0fJ\xf19\xf2\xffI\x0c\x07G\x88\xe6lyre\x94\xc2K\xea|Ir3\xd0\x15\xa8\x9cc\x80\xdbM\xd0\xdf\xc5\x89w\':\x81=]\x0f\xa4j\xbe\x93\x13\xd4\xf1w6\xbf\xb0\x97\xd9\x9c-\n\xa9\xd2\xaf\xac\xdc\xd0\xda9A<\xf8\xee\x8e\x9f\xdd\x8a\x9a\xf8\x94*\xae\x7f\x840a\xb3R9T\xef\x83\x162\xe5pg))\xd7i\x8

In [24]:
r

b'C\x8c\x81\x0b}\x80\xff\x83\xfc\x11\xfe\tv\x92\xf6\xa3\t[\xfc\xbc\xac|\xc0\xbdp\xb2\xdd\xff\xb6\x1a@?\\\xa2|\x07\xf7\xd3\xac3\x9b\xa0\xc8\xf2\xaft\xfb\x1d\xa7gS&\xa3}\xde8<\xee\x91;\x94\xedv\x93kv\x9b"\xa1\x14\xc0\r\xd0\xd6\x80\xfe\x0b\xe3\xd54\n\xc9\tQ\t=1\x0c\t\xc8\x8f\x9d\x80xu\xad\x8a\xbc\xe0,|\x86\x86X\x91e\x1fwW\x13\xc5\xaf,\xcdN\x8e\x91\xcc\x7f\r7!\xbdR\x1f\xaab)q\x04\xc8\xd5.\xac\xb5\x176AF]\xaa\x9f9\xaaR\xa4\xe0\x08\x13|\xc6l\x81\xc0&\xd6\x96\xc6\xc6*2\xb77\x07R\xddJ\xa7\x05\n\x88\x80}\xe3%\xb6f&\x10\x1c\x1cA\x0c\x97\xf1\xac\x01\x9d\xcc\x93\xef\xf0\xad\x9c\xffdw\xceL)ep"\xdb\xd7i\x83o\x81\xc9p\x13NU3S\x82\x90\xb5\xe4%4\xbf\x1c\xaf\x1an2\x83\x1a\x07K\xf9\xaa\x02\x97\x1cA\xe5\x99\x81\xd8\x81\x93W\x07\xf6\x1e\xea\x87$\x1c\xf3\x10\xab5\x86}\x90\xbb\x95\xfe\xc5\xe7K3\xce\xe8\xd7\xf3y\x98\xb1\xb3\xbe\xa8\x19\x00\xb6\x00\xe4\x03\x19_^\x18\xbdO\x03\x1eV$m\xd2\xdf-U\x02Au+\x00.\x94Zk4\xf6\x85\xc3q\xe5x\xf54\x83t\xcf\xf6\xe4nI\x00\x0f\xff\xbb{\xea\x81\xb1\x19\x12\x87\x9c\xd7LM\xc3\xbc.

In [13]:
seek = (meeting.diarizer_seek_timestamp - connection.start_timestamp).total_seconds()

In [14]:
seek

0.0

In [16]:
connection.start_timestamp

datetime.datetime(2024, 5, 28, 15, 11, 31, 469086, tzinfo=tzutc())

In [17]:
meeting.start_timestamp

datetime.datetime(2024, 5, 28, 15, 11, 31, 469086, tzinfo=tzutc())